<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Load Task Table

### Description:
This script comes to demonstrate the usege of task table on a Proteus device.

### set up:
Works on any Proteus device.
It recomended to connect channel 1 and/or 2 to scope to see the waves

### Proteus version: 
Anabelle

In [1]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [2]:
import os
import sys
import gc
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

### Before we start

This example is designed to work both when there is a LAN based communication-service (usually on benchtop/desktop)<br>
And when there's no communication-service, and the user's application has access to hardware-modules in the PXI chassis.

In the first case, when there's a LAN communication, this example uses the class `TEVisaInst` from `tevisainst`.<br>

In the second case, this example uses the classes `TEProteusAdmin` and `TEProteusInst` from `teproteus`.

## Connect to instrument and initialize

In [3]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        admin = TEProteusAdmin()
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(b):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    
def on_button_disconnect_clicked(b):
    print_conn_message('')
    disconnect()

btn_connect = widgets.Button(description="Connect")
btn_disconnect = widgets.Button(description="Disconnect")

btn_connect.on_click(on_button_connect_clicked)
btn_disconnect.on_click(on_button_disconnect_clicked)

display(addr_textbox, btn_connect, btn_disconnect, output)

Text(value='', description='Address:', placeholder='IP Address or Slot Id')

Button(description='Connect', style=ButtonStyle())

Button(description='Disconnect', style=ButtonStyle())

Output()

In [4]:
connect('192.168.0.117')

## Open communication to instrument in chassis

In [5]:
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

# Get the model name
resp = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + resp)

model = resp

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
print(resp)

if resp == 'M0':
    dac_mode = 16 
else:
    dac_mode = 8

print('DAC mode {0} bits'.format(dac_mode))


if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac // 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))


Connected to: Tabor Electronics,P9484M,000000000000,1.235.0 --slot#: 5, 
Model: P9484M
M0
DAC mode 16 bits
0, no error
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,434,432 wave-bytes


In [6]:
FREQ = 9000
inst.send_scpi_cmd(":FREQ:RAST {0}MHz".format(FREQ))
resp = inst.send_scpi_query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:9000000000.0


## Prepare wave-data and markers-data for 12 segments

Prepare three types of waveforms, sine, triangle and square with varying number of cycles. 
Prepare marker for each waveform.


In [7]:
#Set number of waveform segments to build
NumofSegments = 1

waves = [ None for _ in range(NumofSegments)]
marks = [ None for _ in range(NumofSegments)]

#Set segment length
seglen = 2 ** 12 # 1MB
num_cycles = [2 ** n for n in range(NumofSegments)]

if dac_mode == 16:
    seg_wave_bytes = seglen * 2
else:
    seg_wave_bytes = seglen

seg_mark_bytes = seg_wave_bytes // 8

for ii in range(NumofSegments):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    #Prepare Sine wave
    if ii % 3 == 0:
        x = np.linspace(start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
        y = (np.sin(x) + 1.0) * half_dac
        y = np.round(y)
        y = np.clip(y, 0, max_dac)
        if dac_mode == 16:
            waves[ii] = y.astype(np.uint16)
        else:
            waves[ii] = y.astype(np.uint8)
    
    #Prepare triangle waveform
    if ii % 3 == 1:
        x = np.linspace(start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
        y = np.sin(x)
        y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
        y = np.round(y)
        y = np.clip(y, 0, max_dac)
        if dac_mode == 16:
            waves[ii] = y.astype(np.uint16)
        else:
            waves[ii] = y.astype(np.uint8)
    
    #Prepare square waveform
    if ii % 3 == 2:
        x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
        y = np.fmod(x, cyclelen)
        y = (y <= cyclelen / 2) * max_dac
        y = np.round(y)
        y = np.clip(y, 0, max_dac)
        if dac_mode == 16:
            waves[ii] = y.astype(np.uint16)
        else:
            waves[ii] = y.astype(np.uint8)
    
    
    #Prepare marker
    if dac_mode == 16:
        cycle_bytes = cyclelen // 4
    else:
        cycle_bytes = cyclelen // 8
    
    x = np.linspace(start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
    y = np.fmod(x, cycle_bytes)
    y = (y <= cycle_bytes / 2) * 255
    y = np.round(y)
    y = np.clip(y, 0, 255)
    marks[ii] = y.astype(np.uint8)
    
    #
    
    del x, y
    
print('Done')   
 
    


Done


## Download wave-data and markers-data to segments

In [8]:
#num_channels = num_channels/2 if (dac_mode == 8 && 'P908' in model) else num_channels

print(num_channels, dac_mode, model)

if (dac_mode == 8 and 'P908' not in model):
    num_channels = num_channels/2

print(num_channels)

for jj in range(int(num_channels)):
    
    channb = jj+1
    
    for ii in range(NumofSegments):
                
        segnum = 1 + ii
        wav = waves[ii]
        mrk = marks[ii]
        print('Download wave to segment {0} of channel {1}'.format(segnum, channb))

        # Select channel
        cmd = ':INST:CHAN {0}'.format(channb)
        inst.send_scpi_cmd(cmd)

        # Define segment
        cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
        inst.send_scpi_cmd(cmd)  
        # Select the segment
        cmd = ':TRAC:SEL {0}'.format(segnum)
        inst.send_scpi_cmd(cmd)

        # Increase the timeout before writing binary-data:
        inst.timeout = 30000

        # Send the binary-data with *OPC? added to the beginning of its prefix.
        inst.write_binary_data(':TRAC:DATA', wav)


        # Set normal timeout
        inst.timeout = 10000;

        resp = inst.send_scpi_query(':SYST:ERR?')
        resp = resp.rstrip()
        if not resp.startswith('0'):
            print('ERROR: "{0}" after writing binary values'.format(resp))

        print('Download markers to segment {0} of channel {1}'.format(segnum, channb))

        # Increase the timeout before writing binary-data:
        inst.timeout = 30000

        # Send the binary-data with *OPC? added to the beginning of its prefix.
        inst.write_binary_data(':MARK:DATA', mrk)

        # Read the response to the *OPC? query that was added to the prefix of the binary data
        #resp = inst.read()

        # Set normal timeout
        inst.timeout = 10000

        resp = inst.send_scpi_query(':SYST:ERR?')
        resp = resp.rstrip()
        if not resp.startswith('0'):
            print('ERROR: "{0}" after writing binary values'.format(resp))


        # Play the specified segment at the selected channel:
        cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
        inst.send_scpi_cmd(cmd)	    

        # Turn on the output of the selected channel:
        cmd = ':OUTP ON'
        inst.send_scpi_cmd(cmd) 

        resp = inst.send_scpi_query(':SYST:ERR?')
        print(resp)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

del waves
del wav



4 16 P9484M
4
Download wave to segment 1 of channel 1
Download markers to segment 1 of channel 1
ERROR: "219, invalid data size" after writing binary values
219, invalid data size
Download wave to segment 1 of channel 2
Download markers to segment 1 of channel 2
ERROR: "219, invalid data size" after writing binary values
219, invalid data size
Download wave to segment 1 of channel 3
Download markers to segment 1 of channel 3
ERROR: "219, invalid data size" after writing binary values
219, invalid data size
Download wave to segment 1 of channel 4
Download markers to segment 1 of channel 4
ERROR: "219, invalid data size" after writing binary values
219, invalid data size
0, no error


## Load task table

In [9]:
tasklen = 6

for ii in range(int(num_channels)):
    channb=ii+1
    #Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd )
    
    cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
    inst.send_scpi_cmd(cmd )
    
    for jj in range(tasklen):
        curr_task = jj+1
        loop = jj+1
        segnb = jj+1
        cmd = ':TASK:COMP:SEL {0}'.format(curr_task)
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:TYPE SING'
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:LOOP {0}'.format(loop)
        inst.send_scpi_cmd(cmd )
       
        cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
        inst.send_scpi_cmd(cmd )
       
        if curr_task==tasklen:
            cmd = ':TASK:COMP:NEXT1 {0}'.format(1)
            inst.send_scpi_cmd(cmd)
        else:
            cmd = ':TASK:COMP:NEXT1 {0}'.format(curr_task+1)
            inst.send_scpi_cmd(cmd )
    
    cmd = ':TASK:COMP:WRIT'
    inst.send_scpi_cmd(cmd )
    print('Downloading Task table of channel {0}'.format(channb))
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

 

0, no error
0, no error
0, no error
0, no error


## Switch to Task mode and turn on channel and marker outputs

In [10]:
for ii in range(int(num_channels)):
    
    channb=ii+1
    
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    
    cmd = ':OUTP ON'
    inst.send_scpi_cmd(cmd)
    
    #cmd = 'volt MAX';
    #inst.send_scpi_cmd(cmd)
    
    cmd = 'FUNC:MODE TASK'
    inst.send_scpi_cmd(cmd)
    
    if dac_mode==16:
        
        for jj in range(2):
        
            mrk = jj+1
        
            cmd = ':MARK:SEL {0}'.format(mrk)
            inst.send_scpi_cmd(cmd)
        
            cmd = ':MARK ON'
            inst.send_scpi_cmd(cmd)
            
    elif dac_mode==8:
            
        for jj in range(4):

            mrk = jj+1

            cmd = ':MARK:SEL {0}'.format(mrk)
            inst.send_scpi_cmd(cmd)

            cmd = ':MARK ON'
            inst.send_scpi_cmd(cmd)   
            
        

## End

In [11]:
# close connection
disconnect()